In [1]:
import pymongo
from pymongo import MongoClient
from pymongo.mongo_client import MongoClient
from typing import Any

import termcolor

from pymongo.database import Database 
from pymongo.collection import Collection


from bson import SON, raw_bson
from collections.abc import MutableMapping

import pandas as pd
import json
from bson import ObjectId
import os

In [ ]:
# Functions
def get_mongo_client(uri):
def close_mongo_client(client):
def create_database(client, database_name):
def create_collection(database, collection_name):
def insert_data(collection,data):  
def find_data(collection, key_value=""):
def delete_data(collection, key_value=""):
def update_data_entry(collection, filter_criteria, update_data):


In [87]:
class MongoDB_operation:
    
    def __init__(self):
        self.uri=None
        self.client=None
        self.database=None
        self.collection=None
        
    def __str__(self):
        return "MongoDB Object"
    
    def get_mongo_client(self, uri):
        try:
            self.uri=uri
            client=MongoClient(self.uri)
            client.admin.command('ismaster')
            client.server_info()
            if isinstance(client, MongoClient)==False:
                self.client=None
                raise Exception('The object is not a MongoDB client')
            termcolor.cprint("Connected to MongoDB Successfully....",'green',attrs=['bold'])
            self.client=client
            return client
        except Exception as e:
            termcolor.cprint("Error during connection:","red", attrs=['bold'], end=' ')
            print(e)
            
    def create_database(self, database_name):
 
        try:
            client=self.client
            self.database=client[database_name]
            if isinstance(self.database, Database)==False:
                self.database=None
                raise Exception("The object is not a database")
            termcolor.cprint("Database created successfully....",'green',attrs=['bold'])
            return None
        except Exception as e:
            termcolor.cprint("Error creating the database:","red", attrs=['bold'], end=' ')
            print(e) 
            
    def create_collection(self, collection_name):

        try:
            self.collection=self.database[collection_name]
            if isinstance(self.collection, Collection)==False:
                self.collection=None
                raise Exception("The object is not a collection")
            termcolor.cprint("Collection created successfully....",'green',attrs=['bold'])
            return self.collection
        except Exception as e:
            termcolor.cprint("Error creating the collection:","red", attrs=['bold'], end=' ')
            print(e)         
            
    def insert_data(self, data):  

        try:
            if type(data) != list:
                if not isinstance(data, (dict, SON, raw_bson.RawBSONDocument, MutableMapping)):
                    raise Exception("Errrrrrr")

            if type(data) == dict:
                self.collection.insert_one(data)
                termcolor.cprint("One entry inserted successfully....",'green',attrs=['bold'])
            elif type(data) ==list:
                for i,entry in enumerate(data):
                    if type(entry) != dict:
                        dtype=str(type(entry)).split("'")[1]
                        raise Exception(f"Data is not in correct format. \nData type at index {i} of list is '{dtype}', but the entries inside the 'list' should be 'dict' type.")
                self.collection.insert_many(data)
                termcolor.cprint("Data inserted in collection successfully....",'green',attrs=['bold'])
            else:
                dtype=str(type(data)).split("'")[1]
                raise Exception(f"Data is not in correct format. \nThe data type of argument 'data' is '{dtype}', but argument 'data' should pass as 'list' or 'dict' type.")
        except Exception as e:
            termcolor.cprint("Error inserting the entry:","red", attrs=['bold'], end=' ')
            print(e) 
            
    def find_data(self, key_value=""):
#         if isinstance(collection, Collection)==False:
#             print("The object is not a collection")
#             return

        try:    
            if key_value == "":
                item_details=self.collection.find()

            else:
                item_details=self.collection.find(key_value)

            item_list=[]
            for item in item_details:
                item_list.append(item)

            if len(item_list) != 0:
                flag=input("Do you want to print the data as a dataframe(y/n): ").lower()
                if flag=='y':
                    df=pd.DataFrame(item_list)
                    return df
                return item_list
            else:
                termcolor.cprint("Unable to fetch the data..", 'dark_grey', attrs=['bold'])
        except Exception as e:
            termcolor.cprint("Error finding the data:","red", attrs=['bold'], end=' ')
            print(e)     
            
    def delete_data(self, key_value=""):
#         if isinstance(collection, Collection)==False:
#             print("The object is not a collection")
#             return
        try:
            if key_value=="":
                flag=input("It will delete entire data, Do you want to delete the data?(y/n): ").lower()
                if flag=='y':
                    self.collection.delete_many({})
                    print("All entry deleted")
                    return

                else:
                    print("Nothing deleted")
                    return 

            else:
                item_list=[]
                item_details = self.collection.find(key_value)
                for item in item_details:
                    item_list.append(item)
                if len(item_list)==0:
                    termcolor.cprint("Entry doesn't exists of given detail.", 'dark_grey', attrs=['bold'])
                    return

                flag=input("Do you want delete the one entry or mutiple entries?(one/many) ").lower()

                if flag=='one':
                    self.collection.delete_one(key_value)
                    termcolor.cprint("Entry deleted successfully ","green", attrs=['bold'], end=' ')

                elif flag=='many':
                    self.collection.delete_many(key_value)
                    termcolor.cprint("Multiple entries deleted successfully ","green", attrs=['bold'], end=' ')
                else:
                    termcolor.cprint("Incorrect option", "red", attrs=['bold'])

        except Exception as e:
            termcolor.cprint("Error deleting the data:","red", attrs=['bold'], end=' ')
            print(e)
            
            
    def update_data_entry(self, filter_criteria, update_data):
#         if isinstance(collection, Collection)==False:
#             print("The object is not a collection")
#             return
        
        try:
            # Check if the key exists in a document

            item_details=self.collection.find(filter_criteria)

            item_list=[]
            for item in item_details:
                item_list.append(item)

            if len(item_list) == 0:
                raise Exception("Entry does not exist, can not update the data")

            flag=input("Do you want single entry update on multiple?(one/many) ").lower()
            if flag=='one':    
                self.collection.update_one(filter_criteria, update_data)  
                termcolor.cprint("Updated Successfully one entry","green", attrs=['bold'], end=' ')
            elif flag=='many':
                self.collection.update_many(filter_criteria, update_data)
                termcolor.cprint("Updated Successfully multiple entries","green", attrs=['bold'], end=' ')
            else:
                raise Exception("Invalid input")
        except Exception as e:
            termcolor.cprint("Error updating the data:","red", attrs=['bold'], end=' ')
            print(e) 
    
    # Helper function
    def convert_to_serializable(self, obj):
        """Convert MongoDB document to JSON serializable format."""
        if isinstance(obj, ObjectId):
            return str(obj)
        return obj
    
    
    def save_data(self):
        try:
            item_list=list(self.collection.find())
            data=item_list
            flag=input("Do you want to save the data as json file or csv file?(json/csv)").lower()
            file_name=""
            if flag=='json':
                json_file=input("Enter the filename: ")
                data = list(self.collection.find())  # Assuming 'collection' is a MongoDB collection
                serialized_data = json.dumps(data, default=self.convert_to_serializable, indent=2)
                with open(json_file, 'w') as jsonfile:
                    jsonfile.write(serialized_data)
                file_name=json_file
            elif flag=='csv':
                csv_file_name =input("Enter the filename: ")
                df=pd.DataFrame(data)
                df.to_csv(csv_file_name)
                file_name=csv_file_name
                
            else:
                termcolor.cprint("Incorrect option:",'red',attrs=['bold'],end=' ')
                print("choose either 'json' or 'csv'")
                return 
            if os.path.exists(file_name):
                    termcolor.cprint("File:", 'green', attrs=['bold'], end=' ')
                    termcolor.cprint(f"'{file_name}'", 'blue', attrs=['bold'], end=' ')
                    termcolor.cprint("saved successfully....", 'green', attrs=['bold'])
            else:
                raise Exception("File path is not exist")
        except Exception as e:
            termcolor.cprint("Error saving data:",'red',attrs=['bold'],end=' ')
            print(e)
 
    def close_mongo_client(self):
#         if isinstance(client, MongoClient)==False:
#             print('The object is not a MongoDB client')
#             return 

        try:
#             client=self.client
            result=self.client.admin.command('ismaster')
            primary=result.get('ismaster')==True
            secondary=result.get('ismaster')==False
            if primary or secondary or self.client:
                self.client.close()  # This is close of client
                self.database=None
                
                termcolor.cprint("MongoDB client closed successfully..",'magenta',attrs=['bold'])
            else:
                termcolor.cprint("MongoDB client is already closed or not provided.",'dark_grey',attrs=['bold'])

        except Exception as e:
            if str(e)=='Cannot use MongoClient after close':
                termcolor.cprint("MongoDB client is already closed.",'dark_grey',attrs=['bold'])
            else:
                termcolor.cprint("Error closing MongoDB client:","red", attrs=['bold'], end=' ')
                print(e) 
                
    def bulk_insert(self,datafile, collection_name=""):
        """
        Bulk insert data from a CSV, Excel, or JSON file into a MongoDB collection.

        Args:
        - datafile (str): Path to the data file (CSV, Excel, or JSON).
        - collection_name (str, optional): Name of the MongoDB collection. If not provided, the default collection is used.

        Raises:
        - Exception: If an error occurs during the data insertion process.

        Note:
        - The method assumes the MongoDB connection is already established.

        Example:
        ```python
        # If you want to save the data into existing collection
        my_object.bulk_insert('path/to/data.csv')
        
        # If you want to save the data into new collection
        my_object.bulk_insert('path/to/data.csv', 'my_collection')
        ```
        """
        try:
            self.path=datafile
            
            # Configure the type of the file
            if self.path.endswith('.csv'):
                dataframe=pd.read_csv(self.path,encoding='utf-8')

            elif self.path.endswith(".xlsx"):
                dataframe=pd.read_excel(self.path)

            elif self.path.endswith(".json"):
                with open(self.path, 'r') as json_file:
                    datajson = json.load(json_file)

            if not self.path.endswith(".json"):
                datajson=json.loads(dataframe.to_json(orient='records'))
            
            # Set the collection
            if collection_name!="":
                collection=self.create_collection(collection_name)
                
            # call the function to insert the data
            self.insert_data(datajson)
        except Exception as e:
            termcolor.cprint("Error in insert the data:",'red',attrs=['bold'],end=' ')
            print(e)

In [76]:
mongo.bulk_insert('yt.xlsx','col2')

Collection created successfully....
Data inserted in collection successfully....


In [89]:
# Initialize the class object with uri
uri = "localhost:27017"
mongo=MongoDB_operation()

In [90]:
## get mongo client

mongo.get_mongo_client(uri)

Connected to MongoDB Successfully....


MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [91]:
# Create database

database_name="fist1"
mongo.create_database(database_name)

Database created successfully....


In [92]:
# Create collection

collection_name = "colfist1"
mongo.create_collection(collection_name)

Collection created successfully....


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'fist1'), 'colfist1')

In [16]:
import os
import pandas as pd
import json

In [93]:
mongo.save_data()

Do you want to save the data as json file or csv file?(json/csv)csv
Enter the filename: ty.csv
File: 'ty.csv' saved successfully....


In [94]:
# Insert the data

data={"name": "Vishal Verma", "age": 1008, "city": "New York"}
mongo.insert_data(data)

One entry inserted successfully....


In [101]:
# Find the data

# mongo.find_data()


key_value={'name': 'Vishal Verma'}
mongo.find_data(key_value)

Do you want to print the data as a dataframe(y/n): y


,_id,name,age,city,Unnamed: 0
0,65ba0e7bd907c3a471088a7e,Vishal Verma,28,New York,NaN
1,65ba1a31d907c3a471088a81,Vishal Verma,15,New York,NaN
2,65ba2f1beab9e3b26754a98e,Vishal Verma,508,New York,NaN
3,65ba33bfeab9e3b26754a990,Vishal Verma,5898,New York,NaN
4,65ba41b9eab9e3b26754a993,Vishal Verma,25,New York,NaN
5,65ba0e7bd907c3a471088a7e,Vishal Verma,28,New York,4.0
6,65ba1a31d907c3a471088a81,Vishal Verma,18,New York,5.0
7,65ba2f1beab9e3b26754a98e,Vishal Verma,508,New York,8.0
8,65ba33bfeab9e3b26754a990,Vishal Verma,5898,New York,9.0
9,65ba41b9eab9e3b26754a993,Vishal Verma,25,New York,10.0


In [96]:
## Delete the data

w={'age':1008}
mongo.delete_data(w)

Do you want delete the one entry or mutiple entries?(one/many) many
Multiple entries deleted successfully  

In [99]:
# Update the data

filter_criteria = {'age': 55}
update_data = [{'$set': {'age': 36}}]
mongo.update_data_entry(filter_criteria, update_data)

Do you want single entry update on multiple?(one/many) many
Updated Successfully multiple entries 

In [103]:
## Close the mongo client


mongo.close_mongo_client()

MongoDB client is already closed.


In [78]:
class MongoDB_operate:
    
    def __init__(self):
        self.uri=None
        self.client=None
        self.database=None
        self.collection=None
        self.is_closed=True
        
    def __str__(self):
        termcolor.cprint("MongoDB_CRUD Object -","dark_grey", attrs=['bold'], end='\n')
        termcolor.cprint("Connection String:",'dark_grey',attrs=['bold'], end=' ')
        termcolor.cprint(f"{self.uri}",'blue',attrs=['bold'], end='\n')
        termcolor.cprint("Database:",'dark_grey',attrs=['bold'], end=' ')
        termcolor.cprint(f"{None if self.database==None else self.database.name}",'blue',attrs=['bold'], end='\n')
        termcolor.cprint("Collection:",'dark_grey',attrs=['bold'], end=' ')
        termcolor.cprint(f"{None if self.collection==None else self.collection.name}",'blue',attrs=['bold'], end='\n')   
        termcolor.cprint("MongoClient:",'dark_grey',attrs=['bold'], end=' ')
        termcolor.cprint(f"{'Closed' if self.is_closed else 'Open'}",'blue',attrs=['bold'], end='\n')
        return ""     
        
    def get_mongo_client(self, uri):
        """
        Establishes a connection to MongoDB using the provided URI.
        
        Args:
        uri (str): MongoDB connection string.
        
        Returns:
        MongoClient: A MongoDB client object if the connection is successful.

        Raises:
        Exception: If there is an error during the connection process, an exception is raised.
        
        Example Usage:
        ```python
        client = my_object.get_mongo_client(uri)
        ```
        """
        try:
            self.uri=uri
            
            # Attempt to establish a connection to MongoDB
            client=MongoClient(self.uri)
            
            # Check the connection by issuing a command to the 'admin' database
            client.admin.command('ismaster')
            
            # Retrieve server information to further validate the connection
            client.server_info()
            
            # Check if the returned object is an instance of MongoClient
            if isinstance(client, MongoClient)==False:
                self.client=None
                raise Exception('The object is not a MongoDB client')
            
            # Set class attributes for the connected client
            self.client=client
            self.is_closed=False
            
            # Print the success message
            termcolor.cprint("Connected to MongoDB Successfully....",'green',attrs=['bold'])
            
            # Return the connected client
            return client
        
        except Exception as e:
            # Exception handling
            termcolor.cprint("Error during connection:","red", attrs=['bold'], end=' ')
            print(e)        
        
    def create_database(self, database_name):
        """
        Creates a MongoDB database with the specified name.

        Args:
        database_name (str): The name of the database to be created.

        Returns:
        None: Returns None if the database is created successfully.

        Raises:
        Exception: If there is an error during the database creation process, an exception is raised.
        
        Example Usage:
        ```python
        my_object.create_database("my_database")
        ```
        """
        try:
            client=self.client
            
            # Attempt to create a new database with the provided name
            self.database=client[database_name]
            
            if isinstance(self.database, Database)==False:
                self.database=None
                raise Exception("The object is not a database")
            termcolor.cprint("Database created successfully....",'green',attrs=['bold'])
            return None
        except Exception as e:
            termcolor.cprint("Error creating the database:","red", attrs=['bold'], end=' ')
            print(e) 
            
    def create_collection(self, collection_name):
        """
        Creates a MongoDB collection with the specified name within the current database.

        Args:
        collection_name (str): The name of the collection to be created.

        Returns:
        Collection: Returns the created collection object if successful.

        Raises:
        Exception: If there is an error during the collection creation process, an exception is raised.
        
        Example Usage:
        ```python
        # Assume that you have already get the MongoClient and create_collection
        
        collection=my_object.create_collection("my_database")
        ```
        """

        try:
            # Attempt to create a new collection with the provided name within the current database
            self.collection=self.database[collection_name]
            
            if isinstance(self.collection, Collection)==False:
                self.collection=None
                raise Exception("The object is not a collection")
            termcolor.cprint("Collection created successfully....",'green',attrs=['bold'])
            return self.collection
        except Exception as e:
            termcolor.cprint("Error creating the collection:","red", attrs=['bold'], end=' ')
            print(e)         
            
    def insert_data(self, data):  
        """
        Inserts data into the MongoDB collection.

        Args:
        data (dict or list): The data to be inserted. It can be a dictionary for a single document
                             or a list of dictionaries for multiple documents.

        Returns:
        None: Returns None if the data is inserted successfully.

        Raises:
        Exception: If there is an error during the insertion process, an exception is raised,
                   and an error message is printed.
                   
        Example Usage:
        ```python
        data={"name": "abc", "age": 27, "city": "New York"}
        my_object.insert_data(data)
        ```
        """
        try:
            if type(data) != list:
                if not isinstance(data, (dict, SON, raw_bson.RawBSONDocument, MutableMapping)):
                    raise Exception("Data is not in the correct format")
                    
            # Insert the one data entry
            if type(data) == dict:
                self.collection.insert_one(data)
                termcolor.cprint("Inserted successfully(Single entry)....",'green',attrs=['bold'])
                
            # Insert the multiple entries   
            elif type(data) ==list:
                # Verify all the elements of the list as a dict type
                for i,entry in enumerate(data):
                    if type(entry) != dict:
                        dtype=str(type(entry)).split("'")[1]
                        raise Exception(f"Data is not in correct format. \nData type at index {i} of list is '{dtype}', but the entries inside the 'list' should be 'dict' type.")
                self.collection.insert_many(data)
                termcolor.cprint("Data inserted successfully....",'green',attrs=['bold'])
            
            # If the data is not in correct format
            else:
                dtype=str(type(data)).split("'")[1]
                raise Exception(f"Data is not in correct format. \nThe data type of argument 'data' is '{dtype}', but argument 'data' should pass as 'list' or 'dict' type.")
        except Exception as e:
            termcolor.cprint("Error inserting the entry:","red", attrs=['bold'], end=' ')
            print(e) 
            
    def bulk_insert(self,datafile, collection_name=""):
        """
        Bulk insert data from a CSV, Excel, or JSON file into a MongoDB collection.

        Args:
        - datafile (str): Path to the data file (CSV, Excel, or JSON).
        - collection_name (str, optional): Name of the MongoDB collection. If not provided, the default collection is used.

        Raises:
        - Exception: If an error occurs during the data insertion process.

        Note:
        - The method assumes the MongoDB connection is already established.

        Example:
        ```python
        # If you want to save the data into existing collection
        my_object.bulk_insert('path/to/data.csv')
        
        # If you want to save the data into new collection
        my_object.bulk_insert('path/to/data.csv', 'my_collection')
        ```
        """
        try:
            self.path=datafile
            
            # Configure the type of the file
            if self.path.endswith('.csv'):
                dataframe=pd.read_csv(self.path,encoding='utf-8')

            elif self.path.endswith(".xlsx"):
                dataframe=pd.read_excel(self.path)

            elif self.path.endswith(".json"):
                with open(self.path, 'r') as json_file:
                    datajson = json.load(json_file)

            if not self.path.endswith(".json"):
                datajson=json.loads(dataframe.to_json(orient='records'))
            
            # Set the collection
            if collection_name!="":
                collection=self.create_collection(collection_name)
                
            # call the function to insert the data
            self.insert_data(datajson)
        except Exception as e:
            termcolor.cprint("Error in insert the data:",'red',attrs=['bold'],end=' ')
            print(e)
            
    def find_data(self, key_value=""):
        """
        Retrieves data from the MongoDB collection based on the specified key-value pair.

        Args:
        key_value (dict, optional): A dictionary representing the key-value pair to filter the results.
                                    If not provided or an empty dictionary, all documents are retrieved.

        Returns:
        DataFrame or list: Returns a Pandas DataFrame if the user chooses to display the data as a DataFrame,
                           otherwise returns a list of documents.

        Raises:
        Exception: If there is an error during the data retrieval process, an exception is raised,
                   and an error message is printed.
                   
        Example Usage:
        ```python
        # Find all the documents of the collection
        my_object.find_data()
        
        # Find the documents based on key_value
        key_value={'name': 'abc'}
        my_object.find_data(key_value)        
        ```
        """

        try:
            # Find all the documents 
            if key_value == "":
                item_details=self.collection.find()
            
            # Find the documents based on the key_value
            else:
                item_details=self.collection.find(key_value)
            
            # Store documents in list
            item_list=[]
            for item in item_details:
                item_list.append(item)
            
            # If list contains at least one document
            if len(item_list) != 0:
                flag=input("Do you want to print the data as a dataframe(y/n): ").lower()
                if flag=='y':
                    df=pd.DataFrame(item_list)
                    return df
                return item_list
            
            # If list does not contains any document
            else:
                termcolor.cprint("There are no document in the collection or no document for key_value.", 'dark_grey', attrs=['bold'])
        except Exception as e:
            termcolor.cprint("Error finding the data:","red", attrs=['bold'], end=' ')
            print(e)     
            
    def delete_data(self, key_value=""):
        """
        Deletes data from the MongoDB collection based on the specified key-value pair.

        Args:
        key_value (dict, optional): A dictionary representing the key-value pair to identify the documents to delete.
                                    If not provided or an empty dictionary, the user is prompted to delete all entries.

        Returns:
        None: Returns None after the deletion process is completed.

        Raises:
        Exception: If there is an error during the deletion process, an exception is raised,
                   and an error message is printed.
                   
        Example Usage:
        ```python
        # For delete the entire data of collection
        my_object.delete_data()
        
        # Delete the documents based on the key_value
        key_value={'age':38}
        my_object.delete_data(key_value)
        ```
        """

        try:
            # Delete the entire data of collection
            if key_value=="":
                flag=input("It will delete entire data, Do you want to delete the data?(y/n): ").lower()
                if flag=='y':
                    self.collection.delete_many({})
                    termcolor.cprint("All entry deleted",'magenta',attrs=['bold'])
                    return

                else:
                    termcolor.cprint("Nothing deleted")
                    return 
                
            # Delete the documents based on the key_value
            else:
                # Check the documents based on the key value
                item_list = list(self.collection.find(key_value))
                if len(item_list)==0:
                    termcolor.cprint("Entry doesn't exists of given key value.", 'dark_grey', attrs=['bold'])
                    return
                
                # Ask for delete single or multiple entries
                flag=input("Do you want delete the one entry or mutiple entries?(one/many) ").lower()

                if flag=='one':
                    self.collection.delete_one(key_value)
                    termcolor.cprint("Entry deleted successfully ","green", attrs=['bold'], end=' ')

                elif flag=='many':
                    self.collection.delete_many(key_value)
                    termcolor.cprint("Multiple entries deleted successfully ","green", attrs=['bold'], end=' ')
                else:
                    termcolor.cprint("Incorrect option:", "red", attrs=['bold'], end=' ')
                    print("choose either 'one' or 'many'")

        except Exception as e:
            termcolor.cprint("Error deleting the data:","red", attrs=['bold'], end=' ')
            print(e)
            
            
    def update_data_entry(self, filter_criteria, update_data):
        """
        Updates data in the MongoDB collection based on the specified filter criteria.

        Args:
        filter_criteria (dict): A dictionary representing the criteria to identify the documents to update.
        update_data (dict): A dictionary representing the update operation to be applied to matching documents.

        Returns:
        None: Returns None after the update operation is completed.

        Raises:
        Exception: If there is an error during the update process, an exception is raised,
                   and an error message is printed.
                   
        Example Usage:
        ```python
        filter_criteria = {'age': 28}
        update_data = [{'$set': {'age': 55}}]
        my_object.update_data_entry(filter_criteria, update_data)
        ```
        """

        try:
            # Retrieve documents based on the specified filter criteria
            item_list=list(self.collection.find(filter_criteria))
            
            # Raise the error if there is no document based on filter_criteria
            if len(item_list) == 0:
                raise Exception("Entry does not exist, can not update the data")

            # Ask to update single or multiple entries
            flag=input("Do you want single entry update on multiple?(one/many) ").lower()
            
            if flag=='one':    
                self.collection.update_one(filter_criteria, update_data)  
                termcolor.cprint("Updated Successfully (one entry)...","green", attrs=['bold'], end=' ')
            elif flag=='many':
                self.collection.update_many(filter_criteria, update_data)
                termcolor.cprint("Updated Successfully (multiple entries)....","green", attrs=['bold'], end=' ')
            else:
                raise Exception("Invalid input")
        except Exception as e:
            termcolor.cprint("Error updating the data:","red", attrs=['bold'], end=' ')
            print(e) 
    
    def save_data(self):
        """
        Save data from the MongoDB collection to either a JSON or CSV file.

        This method retrieves data from the MongoDB collection and allows the user
        to choose between saving it as a JSON or CSV file. The user is prompted to
        input the desired file format and filename. The data is then serialized and
        saved to the specified file.

        Args:
            No argument requires.

        Returns:
            None: The method doesn't return any value.

        Raises:
            Exception: Raised if there is an error during the file-saving process,
                       or if the specified file path does not exist.

         Example Usage:
        ```python
        my_object.save_data()
        ```
        """
    
        try:
            # Retrieve data from the MongoDB collection
            item_list=list(self.collection.find())
            data=item_list
            
            # Ask the user to save the type of the file
            flag=input("Do you want to save the data as json file or csv file?(json/csv)").lower()
            file_name=""
            
            # Store the file as json format
            if flag=='json':
                json_file=input("Enter the filename: ")
                serialized_data = json.dumps(data, default=self.convert_to_serializable, indent=2)
                with open(json_file, 'w') as jsonfile:
                    jsonfile.write(serialized_data)
                file_name=json_file
                
            # Store the file as csv format
            elif flag=='csv':
                csv_file_name =input("Enter the filename: ")
                df=pd.DataFrame(data)
                df.to_csv(csv_file_name)
                file_name=csv_file_name
                
            else:
                termcolor.cprint("Incorrect option:",'red',attrs=['bold'],end=' ')
                print("choose either 'json' or 'csv'")
                return
            
            # Check if the file is created or not
            if os.path.exists(file_name):
                    termcolor.cprint("File:", 'green', attrs=['bold'], end=' ')
                    termcolor.cprint(f"'{file_name}'", 'blue', attrs=['bold'], end=' ')
                    termcolor.cprint("saved successfully....", 'green', attrs=['bold'])
            else:
                raise Exception("File path is not exist")
                
        except Exception as e:
            termcolor.cprint("Error saving data:",'red',attrs=['bold'],end=' ')
            print(e)
            
            
    def close_mongo_client(self):
        """
        Closes the MongoDB client connection.

        Returns:
        None: Returns None after closing the MongoDB client.

        Raises:
        Exception: If there is an error during the closing process, an exception is raised,
                   and an error message is printed.
        """

        try:
            # Check the status of the MongoDB client connection
            result=self.client.admin.command('ismaster')
            primary=result.get('ismaster')==True
            secondary=result.get('ismaster')==False
            
            # Close the MongoDB client if it is open
            if primary or secondary or self.client:
                self.client.close()
                self.is_closed=True
                termcolor.cprint("MongoDB client closed successfully..",'magenta',attrs=['bold'])
            else:
                termcolor.cprint("MongoDB client is already closed or not provided.",'dark_grey',attrs=['bold'])

        except Exception as e:
            if str(e)=='Cannot use MongoClient after close':
                termcolor.cprint("MongoDB client is already closed.",'dark_grey',attrs=['bold'])
            else:
                termcolor.cprint("Error closing MongoDB client:","red", attrs=['bold'], end=' ')
                print(e) 

In [79]:
mongodb=MongoDB_operate()

In [80]:
print(mongodb)

MongoDB_CRUD Object -
Connection String: localhost:27017
Database: None
Collection: None
MongoClient: Closed



In [81]:
## get mongo client

client=mongodb.get_mongo_client(uri)

Connected to MongoDB Successfully....


In [82]:
# Create database

database_name="fist1"
mongodb.create_database(database_name)

Database created successfully....


In [83]:
# Create collection

collection_name = "colfist1"
collection=mongodb.create_collection(collection_name)

Collection created successfully....


In [109]:
# Insert the data

data={"name": "Vishal Verma", "age": 25, "city": "New York"}
mongodb.insert_data(data)

One entry inserted successfully....


In [86]:
# Find the data

mongodb.find_data()


# key_value={'name': 'Vierma'}
# mongodb.find_data(key_value)

Do you want to print the data as a dataframe(y/n): y


,_id,name,age,city,Unnamed: 0
0,65b2956ab83af4e134e6984d,Abhishek Verma,55,New York,NaN
1,65b357d2a06d2e3de1314856,Abhishek Verma,55,New York,NaN
2,65ba0e7bd907c3a471088a7e,Vishal Verma,28,New York,NaN
3,65ba1a31d907c3a471088a81,Vishal Verma,15,New York,NaN
4,65ba2e22eab9e3b26754a98a,Vishal Verma,1008,New York,NaN
5,65ba2f1beab9e3b26754a98e,Vishal Verma,508,New York,NaN
6,65ba33bfeab9e3b26754a990,Vishal Verma,5898,New York,NaN
7,65ba41b9eab9e3b26754a993,Vishal Verma,25,New York,NaN
8,65bb5ba4a933fb31baf2ed10,Styendra Singh,15,New York,NaN
9,65bb5baba933fb31baf2ed11,Styendra Singh,18,New York,NaN


In [85]:
## Delete the data

w={'age':68}
mongodb.delete_data(w)

Do you want delete the one entry or mutiple entries?(one/many) many
Multiple entries deleted successfully  

In [27]:
# Update the data

filter_criteria = {'age': 18}
update_data = [{'$set': {'age': 15}}]
mongodb.update_data_entry(filter_criteria, update_data)

Do you want single entry update on multiple?(one/many) one
Updated Successfully one entry 

In [95]:
## Close the mongo client

mongodb.close_mongo_client()

MongoDB client closed successfully..
